# 图像分类Pipeline

```{note}
图像分类型任务训练时的pipeline都是类似的，只是模型和数据集不同<br/>
所以我们可以先定义好训练图像分类的函数，模型和数据集作其参数，这样会很方便
```

## 一些辅助函数和类

In [1]:
import torch
import d2l


#@save
def try_gpu():
    """尽量使用gpu"""
    return torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [2]:
#@save
class Accumulator:
    """累计n个数据"""
    
    def __init__(self, n):
        self.data = [0.0] * n

    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [3]:
#@save
def correct_predictions(y_hat, y):
    """
    :param y_hat: (n_samples, n_categories)
    :param y: (n_samples, )
    :return: 正确预测的个数
    """
    y_hat = y_hat.argmax(axis=1)  # across columns
    is_correct = y_hat.type(y.dtype) == y
    return float(is_correct.type(y.dtype).sum())

In [4]:
#@save
def accuracy(net, data_iter, device):
    """
    :param net: 模型
    :param data_iter: 图像分类数据集
    :param device: 尽量使用GPU
    :return: 模型的准确率，这里使用了Accumulator和correct_predictions
    """
    net.eval()  # Set the model to evaluation mode
    metric = d2l.Accumulator(2)  # No. of correct predictions, no. of predictions
    # 预测时需no_grad
    with torch.no_grad():
        for X, y in data_iter:
            X, y = X.to(device), y.to(device)
            # y.numel()表示y中的数据数
            metric.add(d2l.correct_predictions(net(X), y), y.numel())
    return metric[0] / metric[1]

## 训练图像分类的函数

In [5]:
#@save
def train_image_classifier(net, train_iter, test_iter, learning_rate, num_epochs):
    """
    训练图像分类器，记录数据并打印
    e.g. training FashionMNIST
    """
    device = d2l.try_gpu()
    # 需模型和数据均转向device
    net.to(device=device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
    # 记录误差和、正确预测样本数、总样本数
    metric = d2l.Accumulator(3)
    for epoch in range(num_epochs):
        net.train()  # 因为计算accuracy会使net转向eval模式
        metric.reset()
        for x, y in train_iter:
            # Compute prediction error
            x, y = x.to(device), y.to(device)
            y_hat = net(x)
            loss = loss_fn(y_hat, y)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # 记录数据
            metric.add(float(loss) * len(y), d2l.correct_predictions(y_hat, y), y.numel())
        # 打印数据
        print(f"epoch {epoch + 1:<2}, "
              f"loss {metric[0] / metric[2]:.4f}, "
              f"train acc {metric[1] / metric[2]:.4f}, "
              f"test acc {d2l.accuracy(net, test_iter, device):.4f}")

分类问题的损失函数CrossEntropyLoss的计算公式:

$$\mbox{loss}(x, class) = -\mbox{log}\left(\frac{\mbox{exp}(x[class])}{\sum_{j}\mbox{exp}(x[j])}\right) = -x[class] + \log\left ({\sum_{j}\exp({x[j]})}\right )$$